In [ ]:
# Importing necessary libraries
import os      # Library for interacting with the operating system
import time    # Library for tracking the time taken to perform a task
import torch   # PyTorch library for building and training deep learning models
import torchvision   # A PyTorch library containing popular datasets, model architectures, and image transformations
import torchvision.transforms as transforms   # A module containing common image transformations
import torch.nn as nn   # PyTorch library containing various neural network layers and functions
from torch.nn import init   # A function for initializing the weights of a neural network
from torch.optim import lr_scheduler   # A PyTorch module for implementing various learning rate scheduling strategies
import functools   # A module providing various functions for higher-order programming
import numpy as np   # A library for numerical computations
import cv2 as cv   # OpenCV library for image and video processing
from PIL import Image   # A library for image processing and manipulation

In [ ]:
def make_dataset(directory):
    
    images = []  # create an empty list to store the image paths
    
    # check if the given directory is valid or not
    assert os.path.isdir(directory), '%s is not a valid directory' % directory

    # walk through the directory and subdirectories to find all image files
    for root, _, fnames in sorted(os.walk(directory)):
        for fname in fnames:
            path = os.path.join(root, fname)  # get the full path of the image file
            images.append(path)  # add the image path to the list of images
    
    return images  # return the list of image paths

In [ ]:
def __make_power_2(img, base, method=Image.BICUBIC):
    
    """
    Resizes the input image to the nearest size that is a power of 2. If the image size is already a power of 2, then the
    original image is returned. Uses the PIL library for image resizing.

    Args:
        img (PIL.Image): Input image to resize.
        base (int): Base size to round the image dimensions to.
        method (int): Resampling method for image resizing. Default is Image.BICUBIC.

    Returns:
        (PIL.Image): Resized image.
    """
    
    ow, oh = img.size # Get the original width and height of the image.
    h = int(round(oh / base) * base) # Calculate the new height by rounding the original height to the nearest multiple of the base size.
    w = int(round(ow / base) * base) # Calculate the new width by rounding the original width to the nearest multiple of the base size.
    
    # If the new height and width are the same as the original height and width, return the original image.
    if h == oh and w == ow:
        return img

    # Otherwise, print a warning about resizing the image.
    __print_size_warning(ow, oh, w, h)
    
    # Resize the image to the new dimensions using the specified resampling method.
    return img.resize((w, h), method)


In [ ]:
def __print_size_warning(ow, oh, w, h):
    
    if not hasattr(__print_size_warning, 'has_printed'):
        
        print("The image size needs to be a multiple of 4. "
              "The loaded image size was (%d, %d), so it was adjusted to "
              "(%d, %d). This adjustment will be done to all images "
              "whose sizes are not multiples of 4" % (ow, oh, w, h))
        
        __print_size_warning.has_printed = True

In [ ]:
def get_transform(method):
    
    """
    The get_transform function takes in a method argument and returns a PyTorch transform pipeline.

    The transform pipeline consists of three transforms:

    1. A custom lambda transform that resizes the image to the nearest multiple of 4 using the __make_power_2 function.
    2. The ToTensor transform, which converts the image to a PyTorch tensor.
    3. The Normalize transform, which normalizes the tensor values to have a mean of 0.5 and a standard deviation of 0.5 for each channel.
    
    Finally, the Compose transform is used to combine the individual transforms into a single pipeline.
    
    """
    
    # Create a list to hold all the transforms
    transform_list = []
    
    # Add a custom lambda transform that resizes the image to the nearest multiple of 4 using the __make_power_2 function
    transform_list.append(transforms.Lambda(lambda img: __make_power_2(img, base=4, method=method)))

    # Convert the image to a tensor
    transform_list += [transforms.ToTensor()]
    
    # Normalize the image tensor
    transform_list += [transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    
    # Combine the transforms into a single pipeline using the transforms.Compose function and return it
    return transforms.Compose(transform_list)

In [ ]:
def init_net(net, gpu_ids, init_type='normal', init_gain=0.02):

    # Check if any GPU is available and set the network to run on it
    if gpu_ids[0] != -1:
        assert(torch.cuda.is_available())
        net.to(gpu_ids[0])
        net = torch.nn.DataParallel(net, gpu_ids)  # multi-GPUs

    # Function to initialize weights and biases of the network
    def init_func(m): 
        
        classname = m.__class__.__name__
        
        # For Conv and Linear layers, initialize weight with normal distribution and bias with constant 0
        if hasattr(m, 'weight') and (classname.find('Conv') != -1 or classname.find('Linear') != -1):
            init.normal_(m.weight.data, 0.0, init_gain)
            if hasattr(m, 'bias') and m.bias is not None:
                init.constant_(m.bias.data, 0.0)
                
        # For BatchNorm2d layers, initialize weight with normal distribution and bias with constant 0
        elif classname.find('BatchNorm2d') != -1: 
            init.normal_(m.weight.data, 1.0, init_gain)
            init.constant_(m.bias.data, 0.0)

    # Apply the initialized function to the network
    net.apply(init_func)
    
    return net

In [ ]:
def get_scheduler(optimizer, epoch_count, n_epochs, n_epochs_decay):
    
    # Define a lambda function for the learning rate scheduler
    def lambda_rule(epoch):
        lr_l = 1.0 - max(0, epoch + epoch_count - n_epochs) / float(n_epochs_decay + 1)
        return lr_l
    
    # Create a LambdaLR scheduler that uses the lambda_rule function
    scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_rule)
    
    # Return the scheduler object
    return scheduler

In [ ]:
def set_requires_grad(nets, requires_grad=False):
    
    # Check if the input 'nets' is a list or a single network
    if not isinstance(nets, list):
        nets = [nets]
    
    # Loop through all the networks in the list
    for net in nets:
        
        # Check if the current network is not None
        if net is not None:
            
            # Loop through all the parameters of the current network
            for param in net.parameters():
                
                # Set the requires_grad attribute of the current parameter to the specified value
                param.requires_grad = requires_grad

In [ ]:
class ResnetBlock(nn.Module):

    def __init__(self, dim, padding_type, norm_layer, use_dropout, use_bias):
        super(ResnetBlock, self).__init__()
        
        # Build convolutional block
        self.conv_block = self.build_conv_block(dim, padding_type, norm_layer, use_dropout, use_bias)

    def build_conv_block(self, dim, padding_type, norm_layer, use_dropout, use_bias):
        
        # Initialize empty list to store convolutional layers and padding
        conv_block = []
        
        # Set padding size to 0 initially
        p = 0
        
        # Add reflection padding layer to the convolutional block
        conv_block += [nn.ReflectionPad2d(1)]
        
        # Add convolutional layer, normalization layer, and activation function to the convolutional block
        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=p, bias=use_bias), norm_layer(dim), nn.ReLU(True)]
        
        # If use_dropout flag is set to True, add dropout layer to the convolutional block
        if use_dropout:
            conv_block += [nn.Dropout(0.5)]
        
        # Add reflection padding layer to the convolutional block
        conv_block += [nn.ReflectionPad2d(1)]
        
        # Add convolutional layer and normalization layer to the convolutional block
        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=p, bias=use_bias), norm_layer(dim)]

        # Return convolutional block as a sequential layer
        return nn.Sequential(*conv_block)
    
    def forward(self, x):
        
        # Add input tensor x with the convolutional block output
        out = x + self.conv_block(x)  # add skip connections
        
        # Return output tensor
        return out

In [ ]:
class ResnetGenerator(nn.Module):

    def __init__(self, input_nc, output_nc, ngf=64, norm_layer=nn.BatchNorm2d, use_dropout=True, n_blocks=9, padding_type='reflect'):
        # input_nc: number of input channels
        # output_nc: number of output channels
        # ngf: number of filters in the generator's first conv layer
        # norm_layer: normalization layer (default=nn.BatchNorm2d)
        # use_dropout: whether to use dropout layers (default=True)
        # n_blocks: number of ResNet blocks in the generator (default=9)
        # padding_type: type of padding used in conv layers (default='reflect')

        assert(n_blocks >= 0)
        super(ResnetGenerator, self).__init__()

        # By default, don't use bias in conv layers
        use_bias = False

        # Build the generator model using a list to store the layers
        model = [
            nn.ReflectionPad2d(3),  # Pad the input with reflection padding
            nn.Conv2d(input_nc, ngf, kernel_size=7, padding=0, bias=use_bias),  # First convolutional layer
            norm_layer(ngf),  # Apply normalization
            nn.ReLU(True)  # Apply activation function (ReLU)
        ]

        n_downsampling = 2  # Number of downsampling layers
        for i in range(n_downsampling):  # Add downsampling layers
            mult = 2 ** i  # Multiplier for the number of filters
            model += [
                nn.Conv2d(ngf * mult, ngf * mult * 2, kernel_size=3, stride=2, padding=1, bias=use_bias),
                norm_layer(ngf * mult * 2),
                nn.ReLU(True)
            ]

        mult = 2 ** n_downsampling  # Multiplier for the number of filters
        for i in range(n_blocks):  # Add ResNet blocks
            model += [
                ResnetBlock(ngf * mult, padding_type=padding_type, norm_layer=norm_layer, use_dropout=use_dropout, use_bias=use_bias)
            ]

        for i in range(n_downsampling):  # Add upsampling layers
            mult = 2 ** (n_downsampling - i)  # Multiplier for the number of filters
            model += [
                nn.ConvTranspose2d(ngf * mult, int(ngf * mult / 2),
                                   kernel_size=3, stride=2,
                                   padding=1, output_padding=1,
                                   bias=use_bias),
                norm_layer(int(ngf * mult / 2)),
                nn.ReLU(True)
            ]

        # Add final layers to generate output image
        model += [
            nn.ReflectionPad2d(3),  # Pad output with reflection padding
            nn.Conv2d(ngf, output_nc, kernel_size=7, padding=0),  # Last convolutional layer
            nn.Tanh()  # Apply Tanh activation function to generate output image
        ]

        self.model = nn.Sequential(*model)  # Store the generator model as a nn.Sequential module

    def forward(self, input):
        # Forward pass through the generator model

        return self.model(input)

In [ ]:
class NLayerDiscriminator(nn.Module):
    
    def __init__(self, input_nc, ndf=64, n_layers=3, norm_layer=nn.BatchNorm2d):
        super(NLayerDiscriminator, self).__init__()

        use_bias = False  # Whether to use bias in convolutions

        kw = 4  # Kernel size
        padw = 1  # Padding size
        sequence = [nn.Conv2d(input_nc, ndf, kernel_size=kw, stride=2, padding=padw), nn.LeakyReLU(0.2, True)]
        # Add first convolutional layer followed by a leaky ReLU activation function

        nf_mult = 1
        nf_mult_prev = 1
        for n in range(1, n_layers):
            # gradually increase the number of filters by a factor of 2
            nf_mult_prev = nf_mult
            nf_mult = min(2 ** n, 8)
            sequence += [
                nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=2, padding=padw, bias=use_bias),
                # Add a convolutional layer with the specified number of filters, kernel size, stride, padding, and bias
                norm_layer(ndf * nf_mult),
                # Add a normalization layer
                nn.LeakyReLU(0.2, True)
                # Add a leaky ReLU activation function
            ]

        nf_mult_prev = nf_mult
        nf_mult = min(2 ** n_layers, 8)
        sequence += [
            nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=1, padding=padw, bias=use_bias),
            # Add a convolutional layer with the specified number of filters, kernel size, stride, padding, and bias
            norm_layer(ndf * nf_mult),
            # Add a normalization layer
            nn.LeakyReLU(0.2, True)
            # Add a leaky ReLU activation function
        ]

        sequence += [
            nn.Conv2d(ndf * nf_mult, 1, kernel_size=kw, stride=1, padding=padw)
            # Add a convolutional layer that outputs a single-channel prediction map
        ]

        self.model = nn.Sequential(*sequence)

    def forward(self, input):
        # Forward pass of the discriminator network
        return self.model(input)

In [ ]:
class GANLoss(nn.Module):
   
    def __init__(self, gan_mode='lsgan', target_real_label=1.0, target_fake_label=0.0):
 
        super(GANLoss, self).__init__()  # Call the constructor of the parent class

        # Register the target labels as buffers
        self.register_buffer('real_label', torch.tensor(target_real_label))
        self.register_buffer('fake_label', torch.tensor(target_fake_label))
        
        self.gan_mode = gan_mode  # Store the GAN mode (LSGAN, etc.)

        # Create a mean squared error loss function
        self.loss = nn.MSELoss()

    def get_target_tensor(self, prediction, target_is_real):
        # Get the target tensor for the given prediction and target_is_real flag

        if target_is_real:
            # If the target is real, use the real_label buffer
            target_tensor = self.real_label
        else:
            # Otherwise, use the fake_label buffer
            target_tensor = self.fake_label
        # Expand the target tensor to have the same shape as the prediction tensor
        return target_tensor.expand_as(prediction)

    def __call__(self, prediction, target_is_real):
        # Compute the GAN loss for the given prediction and target_is_real flag

        target_tensor = self.get_target_tensor(prediction, target_is_real)
        # Compute the loss using the mean squared error loss function
        loss = self.loss(prediction, target_tensor)
        
        return loss

In [ ]:
class AlignedDataset(torch.utils.data.Dataset):
    """
    A dataset class that reads aligned image pairs from the disk, where images in each pair have the same filename.
    """

    def __init__(self, dataroot, phase):
        """
        Args:
            dataroot (str): Root directory of the dataset.
            phase (str): 'train' or 'test' phase.
        """
        self.dataroot = dataroot
        self.phase = phase
        self.dir_AB = os.path.join(dataroot, phase)  # get the directory of the images
        self.AB_paths = sorted(make_dataset(self.dir_AB))  # get the paths of the images

    def __getitem__(self, index):
        """
        Args:
            index (int): Index of the pair of images.
        Returns:
            A dictionary containing the processed images and their paths.
        """
        # Read an image pair given a random integer index
        AB_path = self.AB_paths[index]
        AB = Image.open(AB_path).convert('RGB')

        # Split AB image into A and B AFTER HAVING BEEN PROCESSED FROM THE SCRIPT "combine_A_and_B.py"
        w, h = AB.size
        w2 = int(w / 2)
        A = AB.crop((0, 0, w2, h))
        B = AB.crop((w2, 0, w, h))

        # Apply the same transform to both A and B
        A_transform = get_transform(Image.BICUBIC)
        B_transform = get_transform(Image.BICUBIC)

        A = A_transform(A)
        B = B_transform(B)

        # Return the processed images and their paths as a dictionary
        return {'A': A, 'B': B, 'A_paths': AB_path, 'B_paths': AB_path}

    def __len__(self):
        """
        Return the total number of image pairs.
        """
        return len(self.AB_paths)

In [ ]:
class CustomDatasetDataLoader():
    
    def __init__(self, dataroot, phase):
        
        # Initialize the class with the given data root and phase
        self.dataroot = dataroot
        self.phase = phase
        
        # Create an instance of the AlignedDataset class using the given data root and phase
        self.dataset = AlignedDataset(dataroot, phase)
        
        # Create a PyTorch DataLoader using the AlignedDataset instance
        self.dataloader = torch.utils.data.DataLoader(
            self.dataset,
            batch_size=1,
            shuffle=False,
            num_workers=0
        )

    def load_data(self):
        
        # Return the instance of the class
        return self

    def __len__(self):
        
        # Return the length of the AlignedDataset instance
        return len(self.dataset)

    def __iter__(self):
        
        # Loop through the DataLoader and yield the data for each batch
        for i, data in enumerate(self.dataloader):
            yield data

In [ ]:
class Pix2PixModel:
    
    """Implementation of the Pix2Pix model."""

    def __init__(self, gpu_ids):
        
        """Constructor method to initialize the model.

        Args:
            gpu_ids (list): List of GPU IDs to use. Use -1 for CPU.
        """
        self.gpu_ids = gpu_ids
        
        # Set the device (GPU or CPU) to use for computations
        self.device = torch.device('cuda:{}'.format(gpu_ids[0])) if gpu_ids[0]!=-1 else torch.device('cpu')

        # Create a list to hold the optimizer instances
        self.optimizers = []
        
        # Create the generator network instance
        self.netG = ResnetGenerator(
            input_nc=3, output_nc=3, ngf=64,
            norm_layer=nn.BatchNorm2d, use_dropout=True, n_blocks=9)
        
        # Initialize the generator network with a normal distribution and a standard deviation of 0.02
        init_net(self.netG, gpu_ids, 'normal', 0.02)
        
        # Create the discriminator network instance
        self.netD = NLayerDiscriminator(
            input_nc=6, ndf=64, n_layers=3, norm_layer=nn.BatchNorm2d)
        
        # Initialize the discriminator network with a normal distribution and a standard deviation of 0.02
        init_net(self.netD, gpu_ids, 'normal', 0.02)

        # Set the GAN loss criterion and move it to the device (GPU or CPU) to use for computations
        self.criterionGAN = GANLoss(gan_mode='lsgan').to(self.device)
        
        # Set the L1 loss criterion
        self.criterionL1 = torch.nn.L1Loss()

        # Create an Adam optimizer instance for the generator network
        self.optimizer_G = torch.optim.Adam(
            self.netG.parameters(), lr=0.0002, betas=(0.5, 0.999))
        
        # Create an Adam optimizer instance for the discriminator network
        self.optimizer_D = torch.optim.Adam(
            self.netD.parameters(), lr=0.0002, betas=(0.5, 0.999))
        
        # Add the optimizer instances to the list
        self.optimizers.append(self.optimizer_G)
        self.optimizers.append(self.optimizer_D)

    def set_input(self, input):
        
        """Set the input data for the model.

        Args:
            input (dict): Dictionary containing the input data. The dictionary should have
                the following keys:
                - A (tensor): The input tensor A.
                - B (tensor): The input tensor B.
                - A_paths (list): List of paths to the A input data.
        """
        
        self.input = input
        
        # Move the input tensor A to the device (GPU or CPU) to use for computations
        self.real_A = input['A'].to(self.device)
        
        # Move the input tensor B to the device (GPU or CPU) to use for computations
        self.real_B = input['B'].to(self.device)
        
        # Get the paths to the input data
        self.image_paths = input['A_paths']

    def forward(self):
        
        """Perform a forward pass through the generator network."""
        
        # Generate fake tensor B from the input tensor A
        self.fake_B = self.netG(self.real_A)


    def backward_D(self):

        # Concatenate real_A and fake_B tensor along the 2nd axis
        fake_AB = torch.cat((self.real_A, self.fake_B), 1) 

        # Pass fake_AB tensor through discriminator network (netD)
        pred_fake = self.netD(fake_AB.detach())

        # Compute the GAN loss for discriminator (loss_D_fake) using the predicted
        # output from discriminator (pred_fake) and the target value of "False"
        self.loss_D_fake = self.criterionGAN(pred_fake, False)

        # Concatenate real_A and real_B tensor along the 2nd axis
        real_AB = torch.cat((self.real_A, self.real_B), 1)

        # Pass real_AB tensor through discriminator network (netD)
        pred_real = self.netD(real_AB)

        # Compute the GAN loss for discriminator (loss_D_real) using the predicted
        # output from discriminator (pred_real) and the target value of "True"
        self.loss_D_real = self.criterionGAN(pred_real, True)

        # Compute the final GAN loss for discriminator (loss_D) by taking the average of
        # loss_D_fake and loss_D_real and multiplying it by 0.5
        self.loss_D = (self.loss_D_fake + self.loss_D_real) * 0.5

        # Compute gradients of loss_D and update parameters of netD
        self.loss_D.backward()

        # Return the losses (loss_D_real, loss_D_fake)
        return (self.loss_D_real, self.loss_D_fake)


    def backward_G(self):

        # Concatenate real_A and fake_B tensor along the 2nd axis
        fake_AB = torch.cat((self.real_A, self.fake_B), 1)

        # Pass fake_AB tensor through discriminator network (netD)
        pred_fake = self.netD(fake_AB)

        # Compute the GAN loss for generator (loss_G_GAN) using the predicted
        # output from discriminator (pred_fake) and the target value of "True"
        self.loss_G_GAN = self.criterionGAN(pred_fake, True)

        # Compute the L1 loss between fake_B and real_B and multiply it by 100.0
        self.loss_G_L1 = self.criterionL1(self.fake_B, self.real_B) * 100.0

        # Compute the final loss for generator (loss_G) by adding loss_G_GAN and loss_G_L1
        self.loss_G = self.loss_G_GAN + self.loss_G_L1

        # Compute gradients of loss_G and update parameters of generator network
        self.loss_G.backward()

        # Return the losses (loss_G_GAN, loss_G_L1)
        return (self.loss_G_GAN, self.loss_G_L1)


    def optimize_parameters(self):

        # Perform forward pass to compute intermediate outputs
        self.forward()              

        # Enable gradient computation for discriminator network (netD)
        set_requires_grad(nets=self.netD, requires_grad=True)       
        
        # Reset gradients of optimizer for discriminator network (optimizer_D)
        self.optimizer_D.zero_grad()     
        
        # Compute the backward pass for discriminator
        rf_losses = self.backward_D()       
        
        # Update the parameters of the discriminator network
        self.optimizer_D.step()    

        # Disable gradient computation for discriminator network (netD)
        set_requires_grad(nets=self.netD, requires_grad=False)  
        
        # Reset gradients of optimizer for generator network (optimizer_G)
        self.optimizer_G.zero_grad()        
       
        gl_losses = self.backward_G()                 
        self.optimizer_G.step() 
        
        return (rf_losses, gl_losses, self.optimizer_G, self.optimizer_D)

In [ ]:
data_loader = CustomDatasetDataLoader('D:/Data Science Projects/CNIC OCR/GANs/pytorch-CycleGAN-and-pix2pix-master/datasets/cnictopix', 'train')
dataset = data_loader.load_data()

In [ ]:
# Initialization of Model

gpu_ids = [-1]
pix2pix = Pix2PixModel(gpu_ids)

In [ ]:
# Set the starting epoch count to 1
epoch_count = 1

# Set the total number of epochs to 1
n_epochs = 1

# Set the number of epochs to linearly decay the learning rate over to 1
n_epochs_decay = 1

# Create a list of schedulers for each optimizer in the 'pix2pix.optimizers' list
# The 'get_scheduler' function is called for each optimizer with the following arguments:
#   - The optimizer object itself
#   - The starting epoch count
#   - The total number of epochs
#   - The number of epochs to decay the learning rate over
# The resulting schedulers are stored in a list called 'schedulers'

schedulers = [get_scheduler(optimizer, epoch_count, n_epochs, n_epochs_decay) for optimizer in pix2pix.optimizers]

In [ ]:
total_iters = 0   # Initialize the total number of iterations to 0

for epoch in range(epoch_count, n_epochs + n_epochs_decay + 1):
    # Loop over the epochs, starting with the current epoch count and going up to n_epochs + n_epochs_decay
    
    epoch_start_time = time.time()  # Record the start time of the current epoch
    
    epoch_iter = 0  # Initialize the number of iterations for the current epoch to 0
    
    print("Epoch Number on Training: ", epoch, '\n')  # Print the current epoch number
    
    old_lr = pix2pix.optimizers[0].param_groups[0]['lr']  # Get the learning rate of the first optimizer in pix2pix
    
    for scheduler in schedulers:  # Loop over the schedulers
        scheduler.step()  # Take a scheduler step to update the learning rates of the optimizers
        
    lr = pix2pix.optimizers[0].param_groups[0]['lr']  # Get the new learning rate of the first optimizer
    
    for i, data in enumerate(dataset):
        # Loop over the data in the dataset, keeping track of the index of the current iteration
        
        total_iters += 1  # Increment the total number of iterations
        
        epoch_iter += 1  # Increment the number of iterations for the current epoch
        
        pix2pix.set_input(data)  # Set the input data for the pix2pix model
        
        (rf_losses, gl_losses, opt_G, opt_D) = pix2pix.optimize_parameters()  # Optimize the parameters of the pix2pix model
        
        losses = [float(gl_losses[0].item()), float(gl_losses[1].item()), float(rf_losses[0].item()), float(rf_losses[1].item())]
        # Get the losses for the current iteration
        
        print('Losses for Epoch number ', epoch, ' for iteration ', i+1 , ' are G_GAN:', losses[0], ', G_L1:', losses[1], ', D_real:', losses[2], ', D_fake:', losses[3], '\n')
        # Print the losses for the current iteration
        
    torch.save({'epoch': epoch,
                'model_G': pix2pix.netG.state_dict(),
                'model_D': pix2pix.netD.state_dict(),
                'optimizer_G_state_dict': opt_G.state_dict(),
                'optimizer_D_state_dict': opt_D.state_dict(),
                'loss_G_GAN': float(gl_losses[0].item()),
                'loss_G_L1': float(gl_losses[1].item()),
                'loss_D_real': float(rf_losses[0].item()),
                'loss_D_fake': float(rf_losses[0].item())}, 'model.pth')
    # Save the current state of the pix2pix model, optimizers, and losses to a file
    
print('Total time taken for training data on ', epoch, 's is: ', time.time() - epoch_start_time)
# Print the total time taken to train the data for all epochs